# Using PySpark ML to Classify Images on IBMs Data Science Experience Local  
By: Michael Travis  
2017-08-21  
This notebook is a basic example of using PySpark ML on 186GB of images to classify 5089 different species of organisms (or 1.2GB and 29 different species). The functions for creating the image features were taken from the book Building Machine Learning Systems With Python by Louis Pedro Coelho and Willi Richert. The data was provided by iNaturalist for a Kaggle competition (https://www.kaggle.com/c/inaturalist-challenge-at-fgvc-2017/data).

# 1. Obtain Images  
Uncomment the img_path based on the amount of data you want to classify. Comment out the other img_path.

In [1]:
#186GB
#img_path = "train_val_images"

#1.2GB
img_path = "train_val_images_mini"

In [2]:
if  not os.path.isdir("{}".format(img_path)):
    if img_path == "train_val_images_mini":
        !wget https://storage.googleapis.com/us_inat_data/train_val/train_val_images_mini.tar.gz
    elif img_path == "train_val_images":
        !wget http://www.vision.caltech.edu/%7Egvanhorn/datasets/inaturalist/fgvc4_competition/train_val_images.tar.gz

In [3]:
if  not os.path.isdir("{}".format(img_path)) and os.path.isdir("{}".format(img_path)):
    if img_path == "train_val_images_mini":
        !tar xzf train_val_images_mini.tar.gz && rm train_val_images_mini.tar.gz
    elif img_path == "train_val_images":
        !tar xzf train_val_images.tar.gz && rm train_val_images.tar.gz

# 2. Libraries  
The cell below imports all the libraries used in this notebook.

In [4]:
#Working with files in directory
from glob import glob

#Data manipulation and handling
import numpy as np
import pandas as pd
import mahotas as mh

#Working with PySpark DataFrames and RDDs
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql.functions import col

#PySpark Machine Learning
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

#Sklearn data handling
from sklearn.model_selection import train_test_split

#To Calculate runtime
import time

# 3. Feature Preperation Functions  
The cells in this section are the functions used to read in the images and compute the features for each one. The features are then saved with their respective label in .csv format to avoid having to repearedly compute the features.

In [5]:
def edginess_sobel(image):
    #Compute "Edginess" of image
    edges = mh.sobel(image, just_filter=True)
    edges = edges.ravel()
    return np.array([np.sqrt(np.dot(edges, edges))])

In [6]:
def chist(im):
    #Compute colour histogram
    #Downsample pixel values:
    im = im // 64

    #Separate RGB channels:
    r,g,b = im.transpose((2,0,1))

    pixels = 1 * r + 4 * g + 16 * b
    hist = np.bincount(pixels.ravel(), minlength=64)
    hist = hist.astype(float)
    return np.log1p(hist)

In [7]:
def surf_im(img_path):
    #Compute Speeded-Up Robust Features
    from sklearn.cluster import KMeans
    from mahotas.features import surf
    
    #Obtain descriptors for all images
    alldescriptors = []
    for im,_ in images(img_path):
        im = mh.imread(im, as_grey=True)
        im = im.astype(np.uint8)
        alldescriptors.append(surf.surf(im, descriptor_only=True))
    
    #Cluster all descriptors
    k = 132
    km = KMeans(k)

    concatenated = np.concatenate(alldescriptors)
    concatenated = concatenated[::64]
    
    km.fit(concatenated)
    
    sfeatures = []
    for d in alldescriptors:
        c = km.predict(d)
        sfeatures.append(np.bincount(c, minlength=k))
    return np.array(sfeatures, dtype=float)

In [8]:
def features_for(im):
    #Compute features for each image
    im = mh.imread(im)
    if len(im.shape) != 3:
        w, h = im.shape[0:2]
        temp = np.empty((w, h, 3), dtype=np.uint8)
        for i in range(3):
            temp[:, :, i] = im
        im = temp
    if im.shape[2] != 3:
        im = im[:,:,0:3]
    img = mh.colors.rgb2grey(im).astype(np.uint8)
    
    return np.concatenate([mh.features.haralick(img).ravel(), chist(im), edginess_sobel(img)])

In [9]:
def images(img_path):
    #Read in each image path
    classes = []
    for image in glob('{}/*/*'.format(img_path)):
        classes.append(image.split('/')[2])
    for ci, cl in enumerate(classes):
        images = glob('{}/*/{}/*.jpg'.format(img_path,cl))
        for im in sorted(images):
            yield im, ci

In [10]:
def create_features(img_path):
    #Create a DataFrame for the features of each image then save it as a .csv file
    #Obtain the local features
    print("Creating global features.")
    lfeatures = []
    labels = []
    for im, ell in images(img_path):
        lfeatures.append(features_for(im))
        labels.append(ell)
    
    #Obtain the global features
    print("Creating local features.")
    all_features = np.concatenate([surf_im(img_path),lfeatures],axis=1)
    all_features = np.array(all_features)
    labels = np.array(labels)

    X = pd.DataFrame(all_features)
    X.columns = X.columns.astype(str)
    y = pd.DataFrame(labels)
    y.columns = ['label']

    data = X.merge(y,how='outer',left_index=True,right_index=True)
    
    #Save DataFrame as .csv file
    print("Saving features as .csv file.")
    if img_path == "train_val_images_mini":
        data.to_csv("imageFeat.csv")
    elif img_path == "train_val_images":
        data.to_csv("imagesFeat186.csv")

# 4. Create Features  
If the .csv file for the features has not yet been created the cell below will use the functions from section 3 to create the .csv file then save it to your working directory.

In [11]:
runtime = time.time()
if img_path == "train_val_images_mini":
    if not os.path.isfile("imageFeat.csv"):
        create_features(img_path)
        
elif img_path == "train_val_images":
    if not os.path.isfile("imageFeat186.csv"):
        create_features(img_path)
        
print("Runtime: {}".format(time.time() - runtime))

Creating global features.
Creating local features.
Saving features as .csv file.
Runtime: 4030.90305281


# 5. Prepare Data for PySpark ML
This section fromats the data from the .csv file to a structure that PySpark ML accepts and splits it into training and testing dataframes.

In [12]:
#Read in .csv file into an RDD
if img_path == "train_val_images_mini":
    rdd = sc.textFile('imageFeat.csv')
elif img_path == "train_val_images":
    rdd = sc.textFile('imageFeat186.csv')

#Format RDD
rdd = rdd.map(lambda line: line.split(","))
header = rdd.first()
rdd = rdd.filter(lambda line:line != header)

#Create a spark session to convert RDD into Spark DataFrame
spark = SparkSession.builder.getOrCreate()
#It seems to be that this brute force method is the only way to do this
df = rdd.map(lambda line: Row(a = float(line[0]), b = float(line[1]), c = float(line[2]), d = float(line[3]), e = float(line[4]), f = float(line[5]), g = float(line[6]), h = float(line[7]), i = float(line[8]), j = float(line[9]),k = float(line[10]), l = float(line[11]), m = float(line[12]), n = float(line[13]), o = float(line[14]), p = float(line[15]), q = float(line[16]), r = float(line[17]), s = float(line[18]), t = float(line[19]),u = float(line[20]), v = float(line[21]), w = float(line[22]), x = float(line[23]), y = float(line[24]), z = float(line[25]), aa = float(line[26]), ab = float(line[27]), ac = float(line[28]), ad = float(line[29]),ae = float(line[30]), af = float(line[31]), ag = float(line[32]), ah = float(line[33]), ai = float(line[34]), aj = float(line[35]), ak = float(line[36]), al = float(line[37]), am = float(line[38]), an = float(line[39]),ao = float(line[40]), ap = float(line[41]), aq = float(line[42]), ar = float(line[43]), at = float(line[44]), au = float(line[45]), av = float(line[46]), aw = float(line[47]), ax = float(line[48]), ay = float(line[49]),az = float(line[50]), ba = float(line[51]), bb = float(line[52]), bc = float(line[53]), bd = float(line[54]), be = float(line[55]), bf = float(line[56]), bg = float(line[57]), bh = float(line[58]), bi = float(line[59]),bj = float(line[60]), bk = float(line[61]), bl = float(line[62]), bm = float(line[63]), bn = float(line[64]), bo = float(line[65]), bp = float(line[66]), bq = float(line[67]), br = float(line[68]), bs = float(line[69]),bt = float(line[70]), bu = float(line[71]), bv = float(line[72]), bw = float(line[73]), bx = float(line[74]), by = float(line[75]), bz = float(line[76]), ca = float(line[77]), cb = float(line[78]), cc = float(line[79]),cd = float(line[80]), ce = float(line[81]), cf = float(line[82]), cg = float(line[83]), ch = float(line[84]), ci = float(line[85]), cj = float(line[86]), ck = float(line[87]), cl = float(line[88]), cm = float(line[89]),cn = float(line[90]), co = float(line[91]), cp = float(line[92]), cq = float(line[93]), cr = float(line[94]), cs = float(line[95]), ct = float(line[96]), cu = float(line[97]), cv = float(line[98]), cw = float(line[99]),cx = float(line[100]), cy = float(line[101]), cz = float(line[102]), da = float(line[103]), db = float(line[104]), dc = float(line[105]), dd = float(line[106]), de = float(line[107]), df = float(line[108]), dg = float(line[109]),dh = float(line[110]), di = float(line[111]), dj = float(line[112]), dk = float(line[113]), dl = float(line[114]), dm = float(line[115]), dn = float(line[116]), do = float(line[117]), dp = float(line[118]), dq = float(line[119]),dr = float(line[120]), ds = float(line[121]), dt = float(line[122]), du = float(line[123]), dv = float(line[124]), dw = float(line[125]), dx = float(line[126]), dy = float(line[127]), dz = float(line[128]), ea = float(line[129]),eb = float(line[130]), ec = float(line[131]), ed = float(line[132]), ee = float(line[133]), ef = float(line[134]), eg = float(line[135]), eh = float(line[136]), ei = float(line[137]), ej = float(line[138]), ek = float(line[139]),el = float(line[140]), em = float(line[141]), en = float(line[142]), eo = float(line[143]), ep = float(line[144]), eq = float(line[145]), er = float(line[146]), es = float(line[147]), et = float(line[148]), eu = float(line[149]),ev = float(line[150]), ew = float(line[151]), ex = float(line[152]), ey = float(line[153]), ez = float(line[154]), fa = float(line[155]), fb = float(line[156]), fc = float(line[157]), fd = float(line[158]), fe = float(line[159]),ff = float(line[160]), fg = float(line[161]), fh = float(line[162]), fi = float(line[163]), fj = float(line[164]), fk = float(line[165]), fl = float(line[166]), fm = float(line[167]), fn = float(line[168]), fo = float(line[169]),fp = float(line[170]), fq = float(line[171]), fr = float(line[172]), fs = float(line[173]), ft = float(line[174]), fu = float(line[175]), fv = float(line[176]), fw = float(line[177]), fx = float(line[178]), fy = float(line[179]),fz = float(line[180]), ga = float(line[181]), gb = float(line[182]), gc = float(line[183]), gd = float(line[184]), ge = float(line[185]), gf = float(line[186]), gg = float(line[187]), gh = float(line[188]), gi = float(line[189]),gj = float(line[190]), gk = float(line[191]), gl = float(line[192]), gm = float(line[193]), gn = float(line[194]), go = float(line[195]), gp = float(line[196]), gq = float(line[197]), gr = float(line[198]), gs = float(line[199]),gt = float(line[200]), gu = float(line[201]), gv = float(line[202]), gw = float(line[203]), gx = float(line[204]), gy = float(line[205]), gz = float(line[206]), ha = float(line[207]), hb = float(line[208]), hc = float(line[209]),hd = float(line[210]), he = float(line[211]), hf = float(line[212]), hg = float(line[213]), hh = float(line[214]), hi = float(line[215]), hj = float(line[216]), hk = float(line[217]), hl = float(line[218]), hm = float(line[219]),hn = float(line[220]), ho = float(line[221]), hp = float(line[222]), hq = float(line[223]), hr = float(line[224]), hs = float(line[225]), ht = float(line[226]), hu = float(line[227]), hv = float(line[228]), hw = float(line[229]),hx = float(line[230]), hy = float(line[231]), hz = float(line[232]), ia = float(line[233]), ib = float(line[234]), ic = float(line[235]), ie = float(line[236]), ig = float(line[237]), ih = float(line[238]), ij = float(line[239]),ik = float(line[240]), il = float(line[241]), im = float(line[242]), io = float(line[243]), ip = float(line[244]), iq = float(line[245]), ir = float(line[246]), it= float(line[247]), iu = float(line[248]), iv = float(line[249]), zlabel = float(line[250]))).toDF()

#Format Spark DataFrame
assembler = VectorAssembler(inputCols=df.columns[:-1], outputCol="features")
output = assembler.transform(df)
data = output.select("features", col("zlabel").alias('label'))

#Unfortunately Spark DataFrame currently does not have a way to stratify the data when splitting it
#so the data will be converted to a Pandas DataFrame to take advantage of sklearns ability to stratify the data
pdData =  data.toPandas()
trainingData, testingData = train_test_split(pdData,test_size=0.8,stratify=pdData[['label']],train_size=0.2,random_state=12)

#Convert Pandas DataFrames back to Spark DataFrames
sqlCtx = SQLContext(sc)
trainingData = sqlCtx.createDataFrame(trainingData)
testingData = sqlCtx.createDataFrame(testingData)

# 6. Spark ML

In [13]:
runtime = time.time()
#Use PySparks Random Forest Classifier
rf = RandomForestClassifier(featuresCol='features', labelCol="label", seed=12)

#To find optimal parameters
paramGrid = ParamGridBuilder()\
                    .addGrid(rf.maxDepth,[10,20,30])\
                    .addGrid(rf.numTrees,[30,35])\
                    .addGrid(rf.impurity,['entropy','gini'])\
                    .build()

evaluator  = MulticlassClassificationEvaluator()

#To crossvalidate the ParamGridBuilder for finding optimal parameters
crossval = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

#Fit Model
model = crossval.fit(trainingData)

print("Runtime: {}".format(time.time() - runtime))

Runtime: 1429.32175899


In [14]:
#Evaluate Model
result = model.transform(testingData)
evaluator.evaluate(result)

0.42385984888545336

# 7. Sources
1. Building Machine Learning Systems With Python by Louis Pedro Coelho and Willi Richert 